In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

from workprojects.rfistats.filterbank import Filterbank
from workprojects.rfistats import analyse_block

In [2]:
### LOAD DATA BLOCK
fname = '/home/vmorello/work/meertrap/SCP/SCP.fil'
fil = Filterbank(fname)

istart = 1000
nsamp = 2000

with open(fil.fname, 'rb') as fobj:
    fobj.seek(fil.sample_offset(istart))
    data = np.fromfile(fobj, dtype=np.float32, count=nsamp*fil.nchans)
    data = data.reshape(nsamp, fil.nchans)

In [3]:
ndata, mask, stats = analyse_block(data)

In [9]:
### Plot normalized data
plt.figure(figsize=(18, 9))
plt.imshow(ndata, cmap=plt.cm.Greys, aspect='auto', vmin=-3, vmax=+6)
plt.tight_layout()
plt.show()

In [12]:
### Channel average power (mean of squares)
cpow = np.mean(ndata**2, axis=0)
cpow_med = np.median(cpow)
cpow_mask, stats = outlier_mask(cpow)

vmin = stats['vmin']
vmax = stats['vmax']


plt.figure(figsize=(18, 5))
plt.plot(cpow)
plt.plot([0, fil.nchans], [vmax, vmax], 'r--')
plt.plot([0, fil.nchans], [vmin, vmin], 'r--')
plt.xlim(0, fil.nchans)
plt.yscale('log')

plt.xlabel('Channel Index')
plt.ylabel('Average Normalized Power')
plt.grid()
#plt.ylim(cpow_med - 3 * nsigma * cpow_rstd, cpow_med + 3 * nsigma * cpow_rstd)
plt.tight_layout()
plt.show()

NameError: name 'outlier_mask' is not defined

In [11]:
from workprojects.rfistats.convolution import BoxcarConvolver
from workprojects.rfistats.block_stats import analyse_segment

### Plot boxcar-convolved data for one channel
ichan = 3357

convolver = BoxcarConvolver(nsamp, wmax=256, wtsp=1.5)
cdata = ndata[ichan] # channel data
stats = analyse_segment(cdata, convolver, thr=6.0)

cprod = stats['conv'] # channel conv. products
cmask = stats['mask'] # channel occupancy mask

plt.figure(figsize=(18, 9))
plt.subplot(211)
plt.imshow(cprod, cmap=plt.cm.Greys, aspect='auto')
plt.ylabel('Boxcar width (samples)', fontsize=12)
plt.yticks(range(convolver.widths.size), convolver.widths)

ax = plt.subplot(212)
plt.plot(cdata, label='Normalized data')
# plt.plot(cprod.max(axis=0), label='Best S/N')
plt.plot(cmask * cdata.max(), linewidth=0)
plt.ylabel('Normalized amplitude', fontsize=12)
ax.fill_between(range(nsamp), cmask * cdata.max(), 0, color='r', alpha=0.2, label='Detected pulses')


plt.grid()
plt.legend(fontsize=12)
plt.xlim(0, nsamp)
plt.xlabel('Time sample index', fontsize=12)
#plt.ylim(cdata.min(), cprod.max() * 1.05)

plt.tight_layout()
plt.show()
